# Final Term - Exercise 1
### David Della Rossa


In [1]:
import cv2

### Task 1

In [51]:
# Create a VideoCapture object and read the frames from an input file
video=cv2.VideoCapture('Exercise1_Files\Traffic_Laramie_1.mp4')

# Check if the video opened successfully
if not video.isOpened(): 
    print("Error opening video file")
    exit(0)

#create an instance of type K-nearest neighbours background subtraction
backSub = cv2.createBackgroundSubtractorKNN(detectShadows = False)

# # Read until video is completed or we press 'q'
while True:
    # Capture frame-by-frame
    # Note that VideoCapture captures the frames of a video without considering the fps of the video
    check, frame = video.read()
    
    if check == False:
        break;
        
    #calculate the foreground mask - this is the frame difference between the original frame and the calculated background
    fgMask = backSub.apply(frame)
    
    # Dilate and erode to get object blobs
    fgMask = cv2.dilate(fgMask, None, 18)
    fgMask = cv2.erode(fgMask, None, 10)
    
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(fgMask,100,255, cv2.THRESH_BINARY)[1]
    
    

    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    
    for c in contours:
        # contourArea() method filters out any small contours
        (x, y, w, h)=cv2.boundingRect(c)
        #filter out contours with a form factor that identifies human shapes
        if (h/w > 1.5)  | (cv2.contourArea(c) < 500):
            continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
    

    if check == True:
    
        cv2.imshow('Webcam', frame)  
        cv2.imshow('Threshold frame', threshold_frame)
        cv2.imshow('FG Mask', fgMask)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

### Task 2

In [31]:
# Create a VideoCapture object and read the frames from an input file
video=cv2.VideoCapture('Exercise1_Files\Traffic_Laramie_1.mp4')

# Check if the video opened successfully
if not video.isOpened(): 
    print("Error opening video file")
    exit(0)

#create an instance of type K-nearest neighbours background subtraction
backSub = cv2.createBackgroundSubtractorKNN(detectShadows = False)

previousBboxes = []
deltaPosX = 10
deltaPosY = 10
deltaSizeH = 10
deltaSizeW = 10

counter = 0

class bBoxMov:
    def __init__(self, x, y, w, h, hor, ver):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.hor = hor
        self.ver = ver

# # Read until video is completed or we press 'q'
while True:
    # Capture frame-by-frame
    # Note that VideoCapture captures the frames of a video without considering the fps of the video
    check, frame = video.read()
    
    if check == False:
        break;
        
    #calculate the foreground mask - this is the frame difference between the original frame and the calculated background
    fgMask = backSub.apply(frame)
    
    # Dilate and erode to get object blobs
    fgMask = cv2.dilate(fgMask, None, 18)
    fgMask = cv2.erode(fgMask, None, 10)
    
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(fgMask,100,255, cv2.THRESH_BINARY)[1]
    
    

    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    currentBboxes = []
    for c in contours:
        # contourArea() method filters out any small contours
        (x, y, w, h)=cv2.boundingRect(c)
        #filter out contours with a form factor that identifies human shapes
        if (h/w > 1.5)  | (cv2.contourArea(c) < 500):
            continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
        currentBboxes.append(bBoxMov(x, y, w, h, 0, 0))
        
    #compare BBoxes;
    
    #foreach BBoxes in currentBBoxes
    #check whether it is found in previousBBoxes
    #if yes, calculate the movement direction and store it
    #if not, it is a new BBox.
    #if a BBox goes out of screen on the left, increase counter.
    #print(len(currentBboxes))
    
    for cBbox in currentBboxes:
        found = False
        for pBbox in previousBboxes:

            deltaX = cBbox.x - pBbox.x
            deltaY = cBbox.y - pBbox.y
            deltaW = cBbox.w - pBbox.w
            deltaH = cBbox.h - pBbox.h
            if (abs(deltaX) > deltaPosX) | (abs(deltaY) > deltaPosY) | (abs(deltaW) > deltaSizeW) |  (abs(deltaH) > deltaSizeH):
                continue
                
            
            #if we are here, it is a match
            found = True
            
                  
            #going out of frame 
            if (cBbox.x < deltaPosX) & (cBbox.hor == -1):
                print(str(cBbox.hor))
                counter+=1
                cBbox.hor = 0
                #print(counter)
                continue
                
            #it's not saving the hor and ver information. I suspect cBbox is created by value
            #If it is so, I need to save it in a separate collection
                
            if cBbox.hor == -1:
                print("Hey")
            #calculate direction on X
            if deltaX > 0:
                cBbox.hor = 1
            elif deltaX < 0:
                cBbox.hor = -1
                    
            else:
                cBbox.hor = 0

            #calculate direction on y
            if deltaY > 0:
                cBbox.ver = 1
            elif deltaY < 0:
                cBbox.ver = -1
            else:
                cBbox.ver = 0
                
        if found == True:
            continue

    
    
    #assign current Bboxes to previous
    previousBboxes = currentBboxes
    

    if check == True:
    
        cv2.imshow('Webcam', frame)  
        cv2.imshow('Threshold frame', threshold_frame)
        cv2.imshow('FG Mask', fgMask)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()